# RAG Application: Ask Questions from a PDF Document using Large Language Models

Retrieval-Augmented Generation (RAG) is a generative AI framework that combines pre-trained large language models (LLMs) with external data sources. RAG improves the output of LLMs by using fresh data from authoritative knowledge bases and enterprise systems to generate more reliable responses.

For example, this project is about using RAG to ask questions from a PDF document. The RAG system uses its large language model to understand the question, then it retrieves relevant information from the PDF document, and finally generates a response. This way, we can extract precise information from a document.

## 0. Setup Ollama

I used [Ollama](https://ollama.com) because it's the easiest way to get up and running with large language models, locally on my computer.

In this case, I used [Llama2](https://llama.meta.com/llama2) model by Meta AI as my choice.

On your terminal, run:

```bash
ollama run llama2
```

## 1. Loading Environment Variables and Setting Up the Model

In [12]:
import os
from dotenv import load_dotenv

# If you want to use the OpenAI API, you need to set the OPENAI_API_KEY environment variable
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"

MODEL = "llama2"

## 2. Prepare Embeddings and Test the Model

In [14]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("what is machine learning in a few words?")

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama2`.

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("what is machine learning in a few words?")

'\nMachine learning is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed.'

## 3. Load the PDF Document

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("openai-and-elon.pdf")
pages = loader.load_and_split()
pages

[Document(page_content="3/10/24, 4:22 PM OpenAI and Elon Musk\nhttps://openai.com/blog/openai-elon-musk 1/13OpenAI and Elon Musk\nWe are dedicated to the OpenAI mission and have pursued it every step of the\nway.\nMarch 5, 2024\nAuthorsBlog\nGreg Brockman\nIlya Sutskever\nJohn Schulman\nSam Altman\nWojciech Zaremba\nOpenAI\nAnnouncements\nThe mission of OpenAI is to ensure AGI benefits all of humanity, which means both building safe and\nbeneficial AGI and helping create broadly distributed benefits. We are now sharing what we've\nlearned about achieving our mission, and some facts about our relationship with Elon. We intend to\nmove to dismiss all of Elon’s claims.", metadata={'source': 'openai-and-elon.pdf', 'page': 0}),
 Document(page_content="3/10/24, 4:22 PM OpenAI and Elon Musk\nhttps://openai.com/blog/openai-elon-musk 2/13We realized building AGI will require far more resources than we’d initially\nimagined\nElon said we should announce an initial $1B funding commitment to OpenA

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""
input_context = input("Enter the context: ")
input_question = input("Enter the question: ")
prompt = PromptTemplate.from_template(template)
print(prompt.format(context=input_context, question=input_question))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## 4. Chain the Prompt, Model, and Parser

In [7]:
chain = prompt | model | parser

In [11]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [9]:
chain.invoke(
    {
        "context": "my specialty is machine learning", 
        "question": "what do you think is my college background?"
    }
)

' Based on the context you provided, I would estimate that your college background is likely in a field related to computer science or engineering, such as computer engineering, electrical engineering, or data science. Your specialty in machine learning suggests that you have a strong foundation in programming and algorithms, as well as statistical analysis and data modeling.'

## 5.0 Use a Vector Database to Store and Retrieve the Results

In [12]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [15]:
questions = [
    "What is OpenAI's mission?",
    "What did they realized?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is OpenAI's mission?
Answer: Based on the provided documents, OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. They aim to build safe and beneficial AGI and help create broadly distributed benefits. OpenAI also wants to move towards dismissing Elon Musk's claims about their relationship with him.

Question: What did they realized?
Answer: From the emails, it seems that Elon Musk and OpenAI realized several things:

1. The potential danger of open-sourcing AI: Ilya Sutskever expressed concerns in an email to Elon Musk about the potential risks of open-sourcing AI, including the possibility of someone unscrupulous with access to overwhelming amount of hardware building an unsafe AI.
2. The importance of scaling AI research: Elon Musk expressed his belief that OpenAI had the potential to reach sustainable Google-scale capital within a decade if they focused on scaling their AI research.
3. The need for significant resources:

## 5.1 Streaming Questions to the Language Model
Basically, what stream does is make the response appear like the style of a chatbot because of a typewriter effect.

In [16]:
for s in chain.stream({"question": "Who are the authors of the blog?"}):
    print(s, end="", flush=True)

Based on the provided context, the answer to the question is: Greg Brockman, Ilya Sutskever, John Schulman, Sam Altman, and Wojciech Zaremba.

## 5.2 Batching Questions to the Language Model

Basically, what batch does is that it allows you to send a batch of questions to the model. This is useful when you have a lot of questions to ask and you don't want to wait for the model to process each question one by one. This is done in parallel.

In [26]:
questions = [
    "tell me the challenges that OpenAI faced",
    #"another question here"
]

In [27]:

chain.batch([{"question": q} for q in questions])

['Based on the email conversation between Elon Musk and Greg Brockman, OpenAI faced several challenges:\n\n1. Financial constraints: OpenAI faced financial difficulties in competing with larger companies like Google, which had a much larger budget for AI research.\n2. Lack of scalability: OpenAI\'s compute horsepower was not sufficient to reach AGI, and it was unclear if the organization could "catch up" to Google scale.\n3. Pressure from investors: OpenAI\'s funding model was not sustainable, and investors might exert too much pressure in the wrong directions.\n4. Competition from Elon Musk: Elon Musk started a competitor to OpenAI, which added an extra layer of competition for the organization.\n5. Balancing open-source contributions with proprietary technology: OpenAI had to balance its mission of making AI beneficial tools widely available with the need to protect its proprietary technology.\n6. Neutrality vs YC-centricity: OpenAI had to navigate the challenge of positioning itself